#  0.0 Introdução 

## 0.1 Problema  de Negócio

Rossmann opera mais de 3.000 drogarias em 7 países europeus. Atualmente, os gerentes de loja da Rossmann têm a tarefa de prever suas vendas diárias com até seis semanas de antecedência. 

**Motivação:** A previsão de vendas foi requisitada pelo CFO em uma reunião mensal sobre os resultados das lojas. 

**Causa Raiz:** Necessidade de realizar reformas nas lojas, necessitando um aporte de dinheiro para concluir essas reformas. 

Assim, deve ser respondido as seguintes perguntas: 
- Qual sera o valor das vendas diarias para as lojas nas proximas 6 semanas? 

## 0.2 Planejamento da Entrega 

1. Resposta 
> previsão das vendas 

2. Formato de entrega
> Json com a previsão para cada loja 

3. Local de entrega 
> API 

## 0.3 Planejamento do Processo 

- Merger dados de treino e features de lojas pelo id da loja. 
- Tratamento de dados faltantes
- Mapa mental de Hipoteses. 
- Criação de novas Features e filtrar variaveis
- EDA
- Modelar dados(Normalização, rescaling, Encoding)
- Selecao de variaveis por importancia(Random Florest) 
- Modelar algoritmo e crosvalidation
- Fine tuning
- Avaliação do algoritmo e Performance de Negocio
- Criar Classe para preparar o dado e aplicar previsao
- Criar API Handler 
- Realizar Deploy

##  0.4 Planejamento de Entrega 

1. Fontes dos Dados
- **Kaggle** https://www.kaggle.com/c/rossmann-store-sales/data

2. Ferramentas 
- Python 3.8 
- jupyter Notebook 
- Pycharm
- sklearn
- coggle

# 1.0 Imports

In [19]:
import pandas as pd
import inflection

## 1.1 Funções de Ajuda

## 1.2 Carregando Dados

In [5]:
df_sales_raw = pd.read_csv('../data/raw/train.csv', low_memory=False)
df_store_raw = pd.read_csv('../data/raw/store.csv', low_memory=False)

In [9]:
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on = 'Store')

# 2.0 Analise Descritiva 

In [16]:
# check point 
df2 = df_raw.copy()

##  2.1 Renomear Colunas

In [20]:
cols_old = df2.columns
snake_case = lambda x : inflection.underscore(x)
cols_new = list(map(snake_case, cols_old))

# rename cols
df2.columns = cols_new

## 2.2 Dimensão dos Dados

O dataset possui 1017209 linhas e 18 colunas. 

In [26]:
df2.shape

(1017209, 18)

## 2.3 Tipo dos Atributos

In [35]:
# verificando o tipo dos atributos
df2.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

In [32]:
# Alterando o tipo da coluna 'date' de OBJECT para DATETIME
df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')

## 2.4 Dados Faltantes(NA)

O dataset possui 6 colunas que contem linhas vazias. (podendo prejudicar a performance dos algoritmos)
- competition_distance
- competition_open_since_month
- competition_open_since_year
- promo2_since_week
- promo2_since_year
- promo_interval

**Por que estao NA? Regra de negocio?**

In [36]:
df2.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64